In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
plotly.tools.set_credentials_file(username='vvr', api_key='Ker155UU70wk3Ccz6iyM')
import plotly.plotly as py
import plotly.graph_objs as go

color = sns.color_palette("husl", 8)

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

import squarify

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
#from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import re



from scipy.sparse import hstack


In [2]:
train= pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = pd.read_csv('campaign_data.csv')

In [3]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [4]:
train.columns 

Index(['id', 'user_id', 'campaign_id', 'send_date', 'is_open', 'is_click'], dtype='object')

In [5]:
train.columns  = ['id', 'userid', 'campaignid', 'send_date', 'open', 'click']

In [6]:
train['open'].value_counts()

0    920401
1    102790
Name: open, dtype: int64

In [7]:
train['click'].value_counts()

0    1010409
1      12782
Name: click, dtype: int64

In [8]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url'],
      dtype='object')

In [9]:
data.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...
2,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...
3,32,Conference,24,19,7,1,\r\n \r\nHi ?\r\n \r\nBefore I dive into why y...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...
4,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
campaign_id             52 non-null int64
communication_type      52 non-null object
total_links             52 non-null int64
no_of_internal_links    52 non-null int64
no_of_images            52 non-null int64
no_of_sections          52 non-null int64
email_body              52 non-null object
subject                 52 non-null object
email_url               52 non-null object
dtypes: int64(5), object(4)
memory usage: 3.7+ KB


In [11]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url'],
      dtype='object')

In [12]:
data['communication_type'].value_counts()

Hackathon          13
Corporate          12
Newsletter          9
Conference          8
Upcoming Events     7
Others              2
Webinar             1
Name: communication_type, dtype: int64

In [13]:
data['campaign_id'].nunique()

52

In [14]:
train.columns

Index(['id', 'userid', 'campaignid', 'send_date', 'open', 'click'], dtype='object')

In [15]:
train['campaignid'].nunique()

26

In [16]:
test['campaign_id'].nunique()

26

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773858 entries, 0 to 773857
Data columns (total 4 columns):
id             773858 non-null object
campaign_id    773858 non-null int64
user_id        773858 non-null int64
send_date      773858 non-null object
dtypes: int64(2), object(2)
memory usage: 23.6+ MB


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023191 entries, 0 to 1023190
Data columns (total 6 columns):
id            1023191 non-null object
userid        1023191 non-null int64
campaignid    1023191 non-null int64
send_date     1023191 non-null object
open          1023191 non-null int64
click         1023191 non-null int64
dtypes: int64(4), object(2)
memory usage: 46.8+ MB


In [19]:
train.columns

Index(['id', 'userid', 'campaignid', 'send_date', 'open', 'click'], dtype='object')

In [20]:
train['campaignid'].nunique()

26

In [21]:
data['campaign_id'].nunique()

52

In [22]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url'],
      dtype='object')

In [23]:
data.head(2)

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...


In [24]:
data[['campaign_id', 'communication_type']].sort_values(['communication_type','campaign_id'])

,campaign_id,communication_type
2,31,Conference
3,32,Conference
7,36,Conference
8,37,Conference
9,38,Conference
11,40,Conference
20,49,Conference
24,53,Conference
12,41,Corporate
14,43,Corporate


In [25]:
data['communication_type'].value_counts()

Hackathon          13
Corporate          12
Newsletter          9
Conference          8
Upcoming Events     7
Others              2
Webinar             1
Name: communication_type, dtype: int64

In [26]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url'],
      dtype='object')

In [27]:
data['email_url'].nunique()

52

In [28]:
data[['campaign_id', 'communication_type','total_links']].sort_values(['communication_type','campaign_id'])

,campaign_id,communication_type,total_links
2,31,Conference,15
3,32,Conference,24
7,36,Conference,13
8,37,Conference,9
9,38,Conference,28
11,40,Conference,10
20,49,Conference,119
24,53,Conference,104
12,41,Corporate,14
14,43,Corporate,11


In [29]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url'],
      dtype='object')

In [30]:
data['subject'].head(3)

0    Sneak Peek: A look at the emerging data scienc...
1    [July] Data Science Expert Meetups & Competiti...
2    Last chance to convince your boss before the E...
Name: subject, dtype: object

In [31]:
data[['campaign_id', 'communication_type','total_links','no_of_images']].sort_values(['communication_type',
                                                                                      'campaign_id', 'total_links'])

,campaign_id,communication_type,total_links,no_of_images
2,31,Conference,15,5
3,32,Conference,24,7
7,36,Conference,13,2
8,37,Conference,9,3
9,38,Conference,28,12
11,40,Conference,10,2
20,49,Conference,119,16
24,53,Conference,104,13
12,41,Corporate,14,2
14,43,Corporate,11,3


In [32]:
import re

data['subject'] = data['subject'].map(lambda x: re.sub(r'\W+', ' ', x))
data['email_body'] = data['email_body'].map(lambda x: re.sub(r'\W+', ' ', x))

In [33]:
def pir(data):
    t = data.text.str.split(expand=True).stack()
    return t.loc[t.str.len() >= 3].groupby(level=0).apply(' '.join)
data['subject'] = data['subject'].str.findall('\w{3,}').str.join(' ')
data['email_body'] = data['email_body'].str.findall('\w{3,}').str.join(' ')

In [34]:
import nltk
data['subject'] = data.apply(lambda row: nltk.word_tokenize(row['subject']), axis=1)
data['email_body'] = data.apply(lambda row: nltk.word_tokenize(row['email_body']), axis=1)


In [35]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

data['subject'] = data["subject"].apply(lambda x: [stemmer.stem(y) for y in x])
data['email_body'] = data["email_body"].apply(lambda x: [stemmer.stem(y) for y in x])


In [36]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['subject'] = data['subject'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))
data['email_body'] = data['email_body'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))

In [37]:
print(data['subject'].iloc[27])

januari year 2018 bigger excit challeng ahead


In [38]:
data['subject'].head()

0              sneak peek look emerg data scienc world
1     juli data scienc expert meetup competit come way
2       last chanc convinc boss befor earli bird expir
3    machin learn reason whi attend datahack summit...
4    delhi ncr firesid chat patil former chief data...
Name: subject, dtype: object

# TFIDF

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(ngram_range = (6,6))
data['subject'] =  v.fit_transform(data['subject'])

In [40]:
data['email_body'] =  v.fit_transform(data['email_body'])

In [41]:
data['subject'].head()

0      (0, 139)\t0.7071067811865476\n  (0, 118)\t0....
1      (0, 139)\t0.7071067811865476\n  (0, 118)\t0....
2      (0, 139)\t0.7071067811865476\n  (0, 118)\t0....
3      (0, 139)\t0.7071067811865476\n  (0, 118)\t0....
4      (0, 139)\t0.7071067811865476\n  (0, 118)\t0....
Name: subject, dtype: object

In [42]:
data.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,29,Newsletter,67,61,12,3,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0....",http://r.newsletters.analyticsvidhya.com/7um44...
1,30,Upcoming Events,18,14,7,1,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0....",http://r.newsletters.analyticsvidhya.com/7up0e...
2,31,Conference,15,13,5,1,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0....",http://r.newsletters.analyticsvidhya.com/7usym...
3,32,Conference,24,19,7,1,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0....",http://r.newsletters.analyticsvidhya.com/7uthl...
4,33,Others,7,3,1,1,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0....",http://r.newsletters.analyticsvidhya.com/7uvlg...


In [43]:
data['email_body'].iloc[20]

<52x3751 sparse matrix of type '<class 'numpy.float64'>'
	with 4300 stored elements in Compressed Sparse Row format>

In [44]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject', 'email_url'],
      dtype='object')

# Label Encoding

In [45]:
le = LabelEncoder()
le.fit(data['communication_type'])
data['communication_type'] = le.transform(data['communication_type'])
#data.head(3)

In [46]:
data.drop('email_url', axis=1, inplace = True)

In [47]:
data.sample()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject
38,67,2,7,3,1,1,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."


# Intersecting

In [48]:
train.columns

Index(['id', 'userid', 'campaignid', 'send_date', 'open', 'click'], dtype='object')

In [49]:
test.columns

Index(['id', 'campaign_id', 'user_id', 'send_date'], dtype='object')

In [50]:
train['id'].nunique()

1023191

In [51]:
train.shape

(1023191, 6)

In [52]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject'],
      dtype='object')

In [53]:
train.columns

Index(['id', 'userid', 'campaignid', 'send_date', 'open', 'click'], dtype='object')

In [54]:
test.columns

Index(['id', 'campaign_id', 'user_id', 'send_date'], dtype='object')

In [55]:
train.head()

,id,userid,campaignid,send_date,open,click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [56]:
train.set_index('campaignid', inplace = True)
train.sample()

,id,userid,send_date,open,click
campaignid,,,,,
54,54_19895,19895,01-12-2017 20:20,0,0


In [57]:
data.columns

Index(['campaign_id', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject'],
      dtype='object')

In [58]:
data.set_index('campaign_id', inplace = True)
data.sample()

,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject
campaign_id,,,,,,,
57,5,40,36,15,4,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."


In [59]:
train.shape

(1023191, 5)

In [60]:
train = pd.merge(train, data, left_index=True, right_index=True, )

In [61]:
train.head()

,id,userid,send_date,open,click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject
29,29_185580,185580,01-07-2017 18:01,0,0,3,67,61,12,3,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."
29,29_103665,103665,01-07-2017 18:05,0,0,3,67,61,12,3,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."
29,29_219721,219721,01-07-2017 18:04,0,0,3,67,61,12,3,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."
29,29_164941,164941,01-07-2017 18:09,0,0,3,67,61,12,3,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."
29,29_159970,159970,01-07-2017 18:03,0,0,3,67,61,12,3,"(0, 1116)\t0.08543576577167614\n (0, 395)\t...","(0, 139)\t0.7071067811865476\n (0, 118)\t0...."


In [62]:
train.shape

(1023191, 12)

In [63]:
test.set_index('campaign_id', inplace = True)

In [64]:
test = pd.merge(test, data, left_index=True, right_index=True, )

In [65]:
test.shape

(773858, 10)

# Model Prepration

# Multi Label Classification

In [66]:
train.columns


Index(['id', 'userid', 'send_date', 'open', 'click', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body', 'subject'],
      dtype='object')

In [67]:
test.columns

Index(['id', 'user_id', 'send_date', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject'],
      dtype='object')

In [68]:
X_train = train.drop(['open', 'click'])
y_train = train[['open', 'click']].copy()
X_train.shape

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



(1023191, 12)

In [69]:
X_test = test
#y_test = test[['open', 'click']].copy()
X_test.shape

(773858, 10)

In [70]:
X_train.columns

Index(['id', 'userid', 'send_date', 'open', 'click', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body', 'subject'],
      dtype='object')

In [71]:
X_test.columns

Index(['id', 'user_id', 'send_date', 'communication_type', 'total_links',
       'no_of_internal_links', 'no_of_images', 'no_of_sections', 'email_body',
       'subject'],
      dtype='object')

In [72]:
X_test.shape

(773858, 10)

In [73]:
X_train.shape

(1023191, 12)

# Model

In [93]:
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


scores = []
class_names = ['open', 'click']
submission = pd.DataFrame.from_dict({'id': test['id']})
#print(X_train.shape)
#print(class_names)
for class_name in class_names:
    train_target = train[class_name]
    
    classifier = LogisticRegression(C=0.1, solver='sag')
    
    cv_score = np.mean(cross_val_score(classifier, X_train, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))
    
    classifier.fit(X_train, train_target)

    
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

(1023191, 12)
['open', 'click']


ValueError: setting an array element with a sequence.

In [96]:
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
print(X_train.shape)
print(class_names)
for class_name in class_names:
    train_target = train[class_name]
    
    classifier = SVC()
    
    cv_score = np.mean(cross_val_score(classifier, X_train, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))
    
    classifier.fit(X_train, train_target)

    
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

(1023191, 12)
['open', 'click']


ValueError: setting an array element with a sequence.

In [ ]:
from sklearn.svm import SVC

In [78]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(X_train, y_train)

# predict
#predictions = classifier.predict(X_test)


from sklearn.model_selection import cross_val_score
scores= cross_val_score(classifier, X_train, y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [ ]:
from sklearn.datasets import make_multilabel_classification

print(X_train.columns)
print(X_train.shape)
print(y_train.shape)
# this will generate a random multi-label dataset
# X_train, y_train = make_multilabel_classification(sparse = True, n_labels = 20,
# return_indicator = 'sparse', allow_unlabeled = False)
 

In [94]:
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(X_train, y_train)

# predict
#predictions = classifier.predict(X_test)


from sklearn.model_selection import cross_val_score
scores= cross_val_score(classifier, X_train, y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [95]:
from sklearn.ensemble import RandomForestClassifier

clf = LabelPowerset(RandomForestClassifier(max_depth=2, random_state=0))
clf.fit(X_train, y_train)

from sklearn.model_selection import cross_val_score
scores= cross_val_score(clf, X_train, y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

X_test, y_test = make_multilabel_classification(sparse = True, n_labels = 20,
return_indicator = 'sparse', allow_unlabeled = False)

predictions = clf.predict(X_test)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [ ]:
y_test= predictions
# print(X_test)
print(y_test)